In [1]:
df = pklload('/Users/ben/data/indradb/2021-08-09/sif.pkl')

In [2]:
from indra.ontology.bio import bio_ontology
bio_ontology.initialize()

INFO: [2021-09-19 20:39:33] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/ben/.indra/bio_ontology/1.17/bio_ontology.pkl


In [3]:
def get_type(ns, id):
    type = bio_ontology.get_type(ns, id)
    if type:
        return type
    if ns == 'CHEBI':
        return 'small_molecule'
    if ns == 'UP':
        return 'nonhuman_gene_protein'
    return None

In [4]:
types = defaultdict(int)
types_evidence = defaultdict(int)
for ans, aid, bns, bid, ec in zip(df['agA_ns'], df['agA_id'], df['agB_ns'], df['agB_id'], df['evidence_count']):
    atype = get_type(ans, aid)
    btype = get_type(bns, bid)
    types[(atype, btype)] += 1
    types_evidence[(atype, btype)] += ec

In [8]:
from indra.util import write_unicode_csv
def dump_counts(fname, counts_dict):
    rows = [['agA_type', 'agB_type', 'count']]
    for (agA_type, agB_type), count in sorted(counts_dict.items(), key=lambda x: x[1], reverse=True):
        rows.append([agA_type, agB_type, count])
    write_unicode_csv(fname, rows)

In [9]:
dump_counts('statements_by_agent_type.csv', types)

In [10]:
dump_counts('statements_by_agent_type_evidence.csv', types_evidence)